# A first practical calculation

We will model a periodic silicon crystal.

| <img src="img/Silicon_crystal.jpg" width=180 height=180 /> |
| --------------- |
| silicon crystal |

To model this in DFTK we run:

In [ ]:
using DFTK

# Look up the structure of silicon in a book
# and define the lattice and the atomic positions
a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
atoms = [Si, Si]
positions = [-ones(3)/8, ones(3)/8]

# Setup an LDA model in DFTK
model = model_LDA(lattice, atoms, positions)

# Discretise the problem in a plane-wave basis
# We use a mesh of 2x2x2 k-Points in the Brillouin zone
# and an energy cutoff of 7 Hartree
basis  = PlaneWaveBasis(model; Ecut=7, kgrid=[2, 2, 2])

# Run the SCF and catch the result:
scfres = self_consistent_field(basis);

This sets up the DFT problem (in this case LDA), constructs an initial guess from the passed structure and then runs an SCF algorithms (details in the next notebooks) to solve it.

The columns of the output indicate the current DFT energy as well as the logarithmic change in energy and density. The last column gives the number of iterations the iterative diagonalisation of the DFT Hamiltonian took.

In the end we obtained the ground-state energy of silicon as

In [ ]:
scfres.energies

We can also look at the density:

In [ ]:
using Plots
heatmap(scfres.ρ[:, :, 5], c=:Blues)

... or access the eigenpairs of the final Hamiltonian diagonalisation
in `scfres.ψ` and `scfres.eigenvalues`:

In [ ]:
scfres.eigenvalues

It might come to a surprise for your why this does not return a flat list,
but in fact a list of two lists. This will be clarified in the following.

Since the perfect silicon crystal is a periodic material,
the Kohn-Sham Hamiltonian $H_\text{KS}$ arising in DFT is symmetric with respect to lattice translations.
See the [next notebook for a derivation](1_self_consistent_field.ipynb).

To avoid the modelling of supercells,
we employ the Bloch ansatz $\psi_{nk}(x) = e^{ikx} u_{nk}(x)$
to $H_\text{KS}$ as discussed before and obtain:

$$\left\{\begin{aligned}
    &\forall k\in\text{BZ}, \ \  1 \leq n \leq N: &\left[
     \frac12 (-i \nabla + k)^2 + \text{diagm}\big( V_\text{ext} + V(\rho) \big) \right] u_{nk} &= \varepsilon_{nk} u_{nk}\\
         &\forall k\in\text{BZ}, \ \ 1 \leq n, m \leq N: &\int_{\Gamma} u_{nk}^\ast u_{mk} &= \delta_{nm} \\
    &&\rho &= \sum_{k\in\text{BZ}} \sum_{n=1}^N f_\text{FD}(\varepsilon_{nk}) |\psi_{nk}|^2
\end{aligned}\right.,$$
where $\Gamma$ denotes the unit cell.

In other words we can diagonalise $H$ $k$-Point by $k$-Point
and only at the level of computing the density we need to
sum over $k$-Points.

In DFTK most quantities in the `scfres` are therefore stored
as a list over $k$-Points. Therefore
```julia
    scfres.eigenvalues[1]
```
gives access to the $\varepsilon_{nk}$ for the first $k$-Point, which by convention is the $\Gamma$ point.

We can also access the DFT Hamiltonian in a convenient way. For example

In [ ]:
ham = scfres.ham;

contains a representation of the full Hamiltonian. Internally it stores representation of individual $k$-point blocks, which behave like a linear operator, e.g. we can multiply it with an array etc.

In [ ]:
hamGamma = scfres.ham.blocks[1];  # Return the (H_k) corresponding to the Γ-point

# Build random vector of a size compatible with hamGamma
A = randn(ComplexF64, size(hamGamma, 2), 1);
A ./= norm(A)  # Normalise
ritz = A' * (hamGamma * A);  # Form Reighlay quotient

This allows to use standard iterative solvers (like LOBPCG) to diagonalise a Hamiltonian. Here we search for `size(A, 2) = 1` eigenpairs.

In [ ]:
diagres = DFTK.lobpcg_hyper(hamGamma, A; prec=PreconditionerTPA(hamGamma.basis, hamGamma.kpoint));
@show diagres.λ diagres.iterations

For debugging it is possible to directly obtain the DFT Hamiltonian as a dense matrix. An alternative way to diagonalise (albeit much slower) is thus:

In [ ]:
matGamma = Array(hamGamma)
eigvals(Hermitian(matGamma), 1:1)

For convenience DFTK also provides a wrapper to diagonalise all k points at once, e.g.

In [ ]:
nev_per_kpoint = 8
eigres = DFTK.diagonalize_all_kblocks(lobpcg_hyper, ham, nev_per_kpoint);
@show eigres.λ[1];  # Eigenvalues at the Gamma point

Similar to the Himiltonian a number of other intrinsic DFT quantities are readily available from DFTK. Moreover we have a convention in the code mapping physical constitiuents of plane-wave DFT (e.g. the block wave $\psi_k$, the basis, the Hamiltonian) consistently to objects in the code. [An overview](https://docs.dftk.org/stable/guide/periodic_problems/#Correspondence-of-theory-to-DFTK-code) is given in the [DFTK documentation](https://docs.dftk.org/stable/guide/periodic_problems/#Correspondence-of-theory-to-DFTK-code).

In our Silicon problem above we used a `kgrid=[2,2,2]` for the BZ discretisation. One would therefore expect 8 $k$-Points
to be present in the discretisation. As it turns out there are only

In [ ]:
length(basis.kpoints)

The reason for this is that the symmetries of the problem allow to make further reductions in effort. If we disable symmetries, we get the expected 8 $k$-Points:

In [ ]:
model_nosym = model_LDA(lattice, atoms, positions, symmetries=false)
length(PlaneWaveBasis(model_nosym; Ecut=15, kgrid=(2, 2, 2)).kpoints)

To conclude this section, let us plot the band structure of silicon,
i.e. the relationship between the eigenvalues of $H$
and the position $k$ inside the Brillouin zone.
(For more details see the previous notebook).

In [ ]:
plot_bandstructure(scfres)